In [1]:
import pandas as pd
import numpy as np

# https://www.kaggle.com/snap/amazon-fine-food-reviews
df = pd.read_csv("./Reviews.csv")
df.info()

#df["Score"].hist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [2]:
del df["Id"], df["ProductId"], df["UserId"], df["ProfileName"], df["HelpfulnessNumerator"], df["HelpfulnessDenominator"], df["Time"], df["Summary"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Score   568454 non-null  int64 
 1   Text    568454 non-null  object
dtypes: int64(1), object(1)
memory usage: 8.7+ MB


In [3]:
from collections import Counter
c = list(df["Score"])
cu = Counter(c)
print(cu)

Counter({5: 363122, 4: 80655, 1: 52268, 3: 42640, 2: 29769})


In [4]:
len_reviews = 29769
print(len_reviews)

len_train = int(round(len_reviews * 0.5))
len_val = (len_reviews - len_train) // 3 * 2
len_test = (len_reviews - len_train) // 3

print(len_train)
print(len_val)
print(len_test)

29769
14884
9922
4961


In [5]:
df_1 = df.loc[df['Score'] == 1].copy()
df_2 = df.loc[df['Score'] == 2].copy()
df_3 = df.loc[df['Score'] == 3].copy()
df_4 = df.loc[df['Score'] == 4].copy()
df_5 = df.loc[df['Score'] == 5].copy()

df_1_train = df_1[:len_train].copy()
df_2_train = df_2[:len_train].copy()
df_3_train = df_3[:len_train].copy()
df_4_train = df_4[:len_train].copy()
df_5_train = df_5[:len_train].copy()

df_1_val = df_1[len_train:len_train+len_val].copy()
df_2_val = df_2[len_train:len_train+len_val].copy()
df_3_val = df_3[len_train:len_train+len_val].copy()
df_4_val = df_4[len_train:len_train+len_val].copy()
df_5_val = df_5[len_train:len_train+len_val].copy()

df_1_test = df_1[len_train+len_val:len_train+len_test+len_val].copy()
df_2_test = df_2[len_train+len_val:len_train+len_test+len_val].copy()
df_3_test = df_3[len_train+len_val:len_train+len_test+len_val].copy()
df_4_test = df_4[len_train+len_val:len_train+len_test+len_val].copy()
df_5_test = df_5[len_train+len_val:len_train+len_test+len_val].copy()

In [6]:
df_train = pd.concat([df_1_train, df_2_train, df_3_train, df_4_train, df_5_train])
df_train = df_train.sample(frac=1)
#df_train["Score"].hist()

In [7]:
df_val = pd.concat([df_1_val, df_2_val, df_3_val, df_4_val, df_5_val])
df_val = df_val.sample(frac=1)
#df_val["Score"].hist()

In [8]:
df_test = pd.concat([df_1_test, df_2_test, df_3_test, df_4_test, df_5_test])
df_test = df_test.sample(frac=1)
#df_test["Score"].hist()

In [9]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import nltk

# nltk.download("stopwords")
# nltk.download('punkt')

stop_words = set(stopwords.words("english"))  

In [10]:
from nltk.stem import PorterStemmer  

stemmer = PorterStemmer()

In [11]:
import spacy
#pip install spacy
#python -m spacy download en
nlp = spacy.load('en_core_web_lg')


In [12]:
print("Before removing stopwords:")
print(df_train.head(n=10))

Before removing stopwords:
        Score                                               Text
185797      3  Product is very grainy........not what I expec...
77873       3  The Switch Black Cherry soda is a fine drink i...
70612       4  I really like Nana's cookies and have placed s...
3830        5  I first saw this product, Panda Licorice, at t...
19710       5  This baby food is both convenient and healthy....
20812       4  I usually buy loose leaf Taylors Darjeeling te...
119238      2  I was really excited when I found these and wa...
20975       5  My wife's favorite tea ever. She was very diss...
185910      3  I am a firm believer that cats should eat good...
3182        5  I am so happy that Amazon got these back in so...


In [13]:
import string

df_train["Text"] = df_train["Text"].apply(lambda row: " ".join([word.lower() for word in word_tokenize(row) if not word.lower() in stop_words and word.lower().isalpha()]))

df_train["Text"] = df_train["Text"].apply(stemmer.stem)

df_train["Text"] = df_train["Text"].apply(lambda row: " ".join([word.lemma_ for word in nlp(row)]))

In [16]:
print("After removing stopwords:")
print(df_train.head(n=10))

After removing stopwords:
        Score                                               Text
185797      3  product grainy expect shipping cost line send ...
77873       3  switch black cherry soda fine drink enjoy carb...
70612       4  really like nana cookie place several order pe...
3830        5  first see product panda licorice memphis tn zo...
19710       5  baby food convenient healthy great worry son e...
20812       4  usually buy loose leaf taylor darjeeling tea h...
119238      2  really excite find want try right away receive...
20975       5  wife favorite tea ever dissapointe local store...
185910      3  firm believer cat eat good meat lot protein av...
3182        5  happy amazon get back could order second case ...


In [17]:
df_val["Text"] = df_val["Text"].apply(lambda row: " ".join([word.lower() for word in word_tokenize(row) if not word.lower() in stop_words and word.lower().isalpha()]))

df_val["Text"] = df_val["Text"].apply(stemmer.stem)

df_val["Text"] = df_val["Text"].apply(lambda row: " ".join([word.lemma_ for word in nlp(row)]))

In [18]:
df_val["Text"] = df_val["Text"].apply(lambda row: " ".join([word.lower() for word in word_tokenize(row) if not word.lower() in stop_words and word.lower().isalpha()]))

df_train["Text"] = df_train["Text"].apply(lambda row: " ".join([word.lower() for word in word_tokenize(row) if not word.lower() in stop_words and word.lower().isalpha()]))

In [19]:
df_train.to_csv("./Review_train.csv")
df_val.to_csv("./Review_val.csv")
df_test.to_csv("./Review_test.csv")